In [22]:
import pandas as pd
import numpy as pd

import pickle
import joblib 

from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import read
from scipy.fftpack import fft
import pydub
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import speech_recognition as sr
import csv
from keras import layers
from keras import models
from sklearn import preprocessing 
from keras.layers.normalization import BatchNormalization
from keras.models import model_from_json
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
#from audioFeatures import bothDF
import ast
import os
from keras.models import load_model

In [2]:
normalized_data = pd.read_csv("./normalized-csv/normalized_data.csv")

In [3]:
normalized_data = {"violin": [normalized_data['violin'][0]]}

In [4]:
normalized_data

{'violin': ['./Medley-solos-DB/Medley-solos-DB_test-7_699396f5-fa59-5ba1-f7f9-2359d11cacd6.wav']}

In [5]:
def samplesList(path_lst):
    '''
    Esta función crea una lista de objetos de AudioSegment
    por cada fichero de la lista que recibe (path_lst),
    después por cada objeto AudioSegment, obtiene la lista
    de samples de cada audio, creando una nueva lista y devolviéndola.
    '''
    audio_lst = []
    audio_samples_lst = []
    for path in path_lst:
        audio = AudioSegment.from_file(path, format='wav')
        audio_lst.append(audio)
    for a in audio_lst:
        audio_samples_lst.append(a.get_array_of_samples())
    
    return audio_samples_lst

def fftList(samples_lst):
    '''
    Esta función recibe una lista de samples de audios,
    hace la transformada de Fourier y devuelve una lista, 
    con toda la lista transformada.
    '''
    fft_lst = []
    for sample in samples_lst:
        four = abs(fft(sample))
        fft_lst.append(four)
    
    return fft_lst

In [6]:
# Cogemos el diccionario y por cada instrumento generamos su lista de samples:

def instSamples(insts_paths):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_samples = {}
    for inst, paths in insts_paths.items():
        inst_samples[inst] = samplesList(paths)
        
    return inst_samples

In [7]:
insts_train_samples = instSamples(normalized_data)

In [9]:
def instFft(insts_samples):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_ffts = {}
    for inst, samples in insts_samples.items():
        abs_four = np.abs(fft(samples))
        inst_ffts[inst] = abs_four
        
    return inst_ffts

insts_ffts = instFft(insts_train_samples)

In [10]:
def dataFramer(insts_ffts):
    '''
    Esta función toma el diccionario que se le pase como argumento
    y, por cada entrada del diccionario, crea un dataframe y los concatena 
    todos.
    '''
    lst_df = []
    for inst, fft_lst in insts_ffts.items():
        inst_tr = {"transformed": [fft for fft in fft_lst], "type": inst}
        lst_df.append(pd.DataFrame(inst_tr))
    return pd.concat(lst_df).reset_index(drop=True)

In [11]:
df_total = dataFramer(insts_ffts)

In [12]:
df_total.head()

,transformed,type
0,"[29651807857.0, 2532744396.219857, 9508085404....",violin


In [13]:
df_total.to_pickle(r'./volin_track_validation.csv')

In [53]:
from keras.models import load_model
new_model = load_model("models/0.9829-accuracy-200000-50epochs-loss0.0823.h5")

In [54]:
X_test=np.vstack(df_total['transformed'])    

In [55]:
X_test.shape
   

(1, 131072)

In [64]:
pred = new_model.predict(X_test)
inst = pred[0].tolist()
print(pred)



[[3.5682092e-06 4.4985895e-06 4.8419565e-06 7.0850168e-05 1.0181603e-06
  1.0566625e-06 6.8041231e-06 9.9990737e-01]]


In [65]:
hola = inst.index(max(inst))
hola

7

In [66]:
print(inst)

[3.568209194781957e-06, 4.498589532886399e-06, 4.841956524614943e-06, 7.085016841301695e-05, 1.0181603329328937e-06, 1.0566625405772356e-06, 6.804123131587403e-06, 0.999907374382019]


In [67]:
len(inst)

8

In [62]:
inst

[3.568209194781957e-06,
 4.498589532886399e-06,
 4.841956524614943e-06,
 7.085016841301695e-05,
 1.0181603329328937e-06,
 1.0566625405772356e-06,
 6.804123131587403e-06,
 0.999907374382019]